In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import os
import pandas as pd
from ast import literal_eval
import textwrap as tw

algo_names = ["tsne", "umap", "trimap", "pacmap"]

In [ ]:
# Load datasets
# Set to True if you want to load the narrative datasets (subjected to copyrigth), False for MNIST only
NARRATIVES = False
if NARRATIVES:
    datasets_names = ["rts", "pdl", "ioc", "mjf"]
else:
    datasets_names = ["mnist"]

In [ ]:
# Load evaluation results
evaluation_results_df = []

N_POINTS = 60000

for dataset in datasets_names:
    results = os.listdir(f"results/{dataset}")
    results = [r for r in results if r.endswith(".csv")]
    for result in results:
        if str(N_POINTS) not in result:
            continue
        print(f"Loading results/{dataset}/{result}")
        evaluation_results_df.append(pd.read_csv(f"results/{dataset}/{result}"))

print(f"Loaded {len(evaluation_results_df)} evaluation results")
evaluation_results_df = pd.concat(evaluation_results_df)
evaluation_results_df.rename(columns={"index": "dataset_algo"}, inplace=True)

evaluation_results_df["rta_local"] = evaluation_results_df["rta"].map(lambda x: eval(x)["local"][0])
evaluation_results_df["rta_global"] = evaluation_results_df["rta"].map(lambda x: eval(x)["global"][0])
evaluation_results_df["rta_all"] = evaluation_results_df["rta"].map(lambda x: eval(x)["all"][0])
evaluation_results_df.drop(columns=["rta"], inplace=True)

evaluation_results_df["dist_corr"] = evaluation_results_df["dist_corr"].map(lambda x: eval(x)[0])

evaluation_results_df.head()

In [ ]:
scalar_metrics = ["neighbor_kept_ratio", "global_score", "rta_local", "rta_global", "rta_all", "dist_corr", "run_time"]

scalar_metrics_df = evaluation_results_df[["dataset_algo"] + scalar_metrics]
scalar_metrics_df["run"] = scalar_metrics_df["dataset_algo"].apply(lambda x: x.split("_")[2].replace("run", "")).astype(int)
scalar_metrics_df["dataset_algo"] = scalar_metrics_df["dataset_algo"].apply(lambda x: x.split("_")[0] + "_" + x.split("_")[1])
scalar_metrics_df = scalar_metrics_df.groupby(["dataset_algo"])[scalar_metrics].agg(["mean", "std"])
scalar_metrics_df.columns = ['_'.join(col).strip() for col in scalar_metrics_df.columns.values]
scalar_metrics_df = scalar_metrics_df.apply(lambda row: [(row[f"{metric}_mean"], row[f"{metric}_std"]) for metric in scalar_metrics], axis=1, result_type='expand')
scalar_metrics_df.columns = scalar_metrics
scalar_metrics_df.reset_index(inplace=True)
scalar_metrics_df["dataset"] = scalar_metrics_df["dataset_algo"].apply(lambda x: x.split("_")[0])
scalar_metrics_df["algo"] = scalar_metrics_df["dataset_algo"].apply(lambda x: x.split("_")[1])
scalar_metrics_df.head()

In [ ]:
# Round all values to 3 decimal places
scalar_metrics_df_to_save = scalar_metrics_df.copy()
for metric in scalar_metrics:
    scalar_metrics_df_to_save[metric] = scalar_metrics_df_to_save[metric].apply(lambda x: (round(x[0], 3), round(x[1], 3)))
scalar_metrics_df_to_save.to_csv("results/scalar_metrics_sample60k.csv")

In [ ]:
barwidth = 0.2
n_datasets = len(scalar_metrics_df["dataset"].unique())
n_algos = len(scalar_metrics_df["algo"].unique())
colors = plt.cm.get_cmap("tab10")
legend_patches = [mpatches.Patch(color=colors(i), label=algo.upper()) for i, algo in enumerate(algo_names)]

# Plot neighborhood preservation on single plot
fig, ax = plt.subplots(figsize=(10, 4))
for i, dataset in enumerate(datasets_names):
    for j, algo in enumerate(algo_names):
        df = scalar_metrics_df[(scalar_metrics_df["dataset"] == dataset) & (scalar_metrics_df["algo"] == algo)]
        mean,std = df["neighbor_kept_ratio"].values[0]
        ax.bar(i + j * barwidth, mean, yerr=std, width=barwidth, label=f"{algo}", color = colors(j))
ax.set_xticks(np.arange(n_datasets) + barwidth * (n_algos - 1) / 2)
ax.set_xticklabels([w.upper() for w in datasets_names], fontsize = 12, fontweight='bold')
ax.set_ylabel("Neighborhood Kept Ratio", fontsize = 12, fontweight='bold')
ax.legend(handles=legend_patches, loc="upper left", bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
# Plot global score
fig, ax = plt.subplots(figsize=(10, 4))
for i, dataset in enumerate(datasets_names):
    for j, algo in enumerate(algo_names):
        df = scalar_metrics_df[(scalar_metrics_df["dataset"] == dataset) & (scalar_metrics_df["algo"] == algo)]
        mean,std = df["global_score"].values[0]
        ax.bar(i + j * barwidth, mean, yerr=std, width=barwidth, label=f"{algo}", color = colors(j))
ax.set_xticks(np.arange(n_datasets) + barwidth * (n_algos - 1) / 2)
ax.set_xticklabels([w.upper() for w in datasets_names], fontsize = 12, fontweight='bold')
ax.set_ylabel("Global Score", fontsize = 12, fontweight='bold')
ax.legend(handles=legend_patches, loc="upper left", bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
# Plot random triplet accuracy
fig, axs = plt.subplots(3, 1, figsize=(10, 8))
for i, dataset in enumerate(datasets_names):
    for j, algo in enumerate(algo_names):
        df = scalar_metrics_df[(scalar_metrics_df["dataset"] == dataset) & (scalar_metrics_df["algo"] == algo)]
        # Local RTA
        mean,std = df["rta_local"].values[0]
        axs[0].bar(i + j * barwidth, mean, yerr=std, width=barwidth, label=f"{algo}", color = colors(j))
        # Global RTA
        mean,std = df["rta_global"].values[0]
        axs[1].bar(i + j * barwidth, mean, yerr=std, width=barwidth, label=f"{algo}", color = colors(j))
        # All RTA
        mean,std = df["rta_all"].values[0]
        axs[2].bar(i + j * barwidth, mean, yerr=std, width=barwidth, label=f"{algo}", color = colors(j))

for ax,label in zip(axs,["Local", "Global", "All"]):
    ax.set_xticks(np.arange(n_datasets) + barwidth * (n_algos - 1) / 2)
    ax.set_xticklabels([w.upper() for w in datasets_names], fontsize = 12, fontweight='bold')
    ax.set_ylabel(label, fontsize = 12, fontweight='bold')
plt.suptitle("Random Triplet Accuracy", fontsize = 14, fontweight='bold')
axs[0].legend(handles=legend_patches, loc="upper left", bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
# Plot Random Triplet Accuracy (all)
fig, ax = plt.subplots(figsize=(10, 4))
for i, dataset in enumerate(datasets_names):
    for j, algo in enumerate(algo_names):
        df = scalar_metrics_df[(scalar_metrics_df["dataset"] == dataset) & (scalar_metrics_df["algo"] == algo)]
        mean,std = df["rta_all"].values[0]
        ax.bar(i + j * barwidth, mean, yerr=std, width=barwidth, label=f"{algo}", color = colors(j))
ax.set_xticks(np.arange(n_datasets) + barwidth * (n_algos - 1) / 2)
ax.set_xticklabels([w.upper() for w in datasets_names], fontsize = 12, fontweight='bold')
ax.set_ylabel("Random Triplet Accuracy", fontsize = 12, fontweight='bold')
ax.legend(handles=legend_patches, loc="upper left", bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
# Plot Spearman correlation of distances
fig, ax = plt.subplots(figsize=(10, 4))
for i, dataset in enumerate(datasets_names):
    for j, algo in enumerate(algo_names):
        df = scalar_metrics_df[(scalar_metrics_df["dataset"] == dataset) & (scalar_metrics_df["algo"] == algo)]
        mean,std = df["dist_corr"].values[0]
        ax.bar(i + j * barwidth, np.abs(mean), yerr=std, width=barwidth, label=f"{algo}", color = colors(j))
ax.set_xticks(np.arange(n_datasets) + barwidth * (n_algos - 1) / 2)
ax.set_xticklabels([w.upper() for w in datasets_names], fontsize = 12, fontweight='bold')
ax.set_ylabel("Distances Spearman Correlation", fontsize = 12, fontweight='bold')
ax.legend(handles=legend_patches, loc="upper left", bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
# Single plot for all scalar metrics, in a column
metrics_to_plot = ["neighbor_kept_ratio", "global_score", "rta_all", "dist_corr"]
metrics_labels = ["Neighborhood Kept Ratio", "Global Score", "Random Triplet Accuracy", "Distances Spearman Correlation"]
fig, axs = plt.subplots(len(metrics_to_plot) + 1, 1, figsize=(6, 10))
for i, metric in enumerate(metrics_to_plot):
    for j, dataset in enumerate(datasets_names):
        for k, algo in enumerate(algo_names):
            df = scalar_metrics_df[(scalar_metrics_df["dataset"] == dataset) & (scalar_metrics_df["algo"] == algo)]
            mean,std = df[metric].values[0]
            axs[i+1].bar(j + k * barwidth, mean, yerr=std, width=barwidth, label=f"{algo}", color = colors(k))
    axs[i+1].set_xticks(np.arange(n_datasets) + barwidth * (n_algos - 1) / 2)
    axs[i+1].set_xticklabels([w.upper() for w in datasets_names], fontsize = 12, fontweight='bold')
    axs[i+1].set_ylabel(tw.fill(metrics_labels[i], 15), fontsize = 12, fontweight='bold')
plt.suptitle("Scalar Metrics", fontsize = 14, fontweight='bold')
axs[0].legend(handles=legend_patches, loc="lower left", fontsize = 14)
axs[0].axis("off")
plt.tight_layout()
plt.show()

## Metrics at rank k

In [ ]:
metrics_at_k = ["t_values", "c_values", "lcmc_values"]
at_k_metrics = evaluation_results_df[["dataset_algo"] + metrics_at_k]
at_k_metrics["dataset"] = at_k_metrics["dataset_algo"].apply(lambda x: x.split("_")[0])
at_k_metrics["algo"] = at_k_metrics["dataset_algo"].apply(lambda x: x.split("_")[1])
at_k_metrics["run"] = at_k_metrics["dataset_algo"].apply(lambda x: x.split("_")[2].replace("run", "")).astype(int)

at_k_metrics.head()

In [ ]:
k_values = np.arange(0,len(at_k_metrics["t_values"].values[0]))
k_metrics_handles = []
k_metrics_colors = plt.cm.get_cmap("tab20", len(algo_names) * len(datasets_names))

fig, axs = plt.subplots(3, len(datasets_names), figsize=(16, 8))

for d,dataset in enumerate(datasets_names):
    for i,algo in enumerate(algo_names):
        k_metrics_handles.append(f"{dataset}_{algo}")
        df = at_k_metrics[(at_k_metrics["dataset"] == dataset) & (at_k_metrics["algo"] == algo)]
        t_values = df["t_values"].values
        c_values = df["c_values"].values
        lcmc_values = df["lcmc_values"].values
        for j in range(len(t_values)):
            axs[0,d].scatter(k_values, t_values[j], color=colors(i), linestyle='-', marker='x', s=1)
            axs[1,d].scatter(k_values, c_values[j], color=colors(i), linestyle='-', marker='x', s=1)
            axs[2,d].scatter(k_values, lcmc_values[j], color=colors(i), linestyle='-', marker='x', s=1)
        
plt.suptitle("Metrics at K", fontsize = 14, fontweight='bold')
axs[0,0].set_ylabel("Trustworthiness", fontsize = 12, fontweight='bold')
axs[1,0].set_ylabel("Continuity", fontsize = 12, fontweight='bold')
axs[2,0].set_ylabel("LCMC", fontsize = 12, fontweight='bold')
# Shared x axis labels
for ax in axs.flat:
    ax.set_xlabel("K", fontsize = 12, fontweight='bold')

axs[0,3].legend(handles=legend_patches, loc="upper left", bbox_to_anchor=(1, 1))
plt.tight_layout()
plt.show()